### Detecting Collisions by Classifying the SSP

This notebook demonstrates how to train an SVM (with RBF kernel) to detect whether two objects occupy the same physical location within a single SSP (i.e., a binary classifier where "P" means "collision" and "N" means "no collision").

Performance is demonstrated in four ways:
 - Confusion matrix on training data.
 - Confusion matrix on validation data.
 - Detecting a collision when two objects travel into another along a linear path.
 - Detecting a number of collisions when two objects travel independently along two lemniscate curves.
 
For the final two demonstrations, the animation switches polarity to visually indicate a collision.

In [ ]:
%matplotlib inline

In [ ]:
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from IPython.display import HTML

from ssp.maps import Spatial2D
from ssp.plots import heatmap_animation, create_gif

In [ ]:
dim = 768
names = ["A", "B"]
map_radius = 5  # half-length of square centered about (0, 0)

In [ ]:
ssp_map = Spatial2D(dim=dim, scale=1, rng=np.random.RandomState(seed=0))
ssp_map.build_grid(x_len=map_radius, y_len=map_radius,
                   x_spaces=101, y_spaces=101, centered=True)

In [ ]:
def generate_collision_data(vocab, objs, low, high, n_samples=2048):
    rng = vocab.pointer_gen.rng
    pos = rng.uniform(low=low, high=high, size=(n_samples, len(objs), 2))
    x = np.zeros((n_samples, vocab.dimensions), dtype=np.float64)
    dist = np.zeros((n_samples,), dtype=np.float64)
    for i in range(n_samples):
        x[i, :] = np.sum(
            [
                vocab[objs[j]]
                * vocab["X"] ** pos[i, j, 0]
                * vocab["Y"] ** pos[i, j, 1]
                for j in range(len(objs))
            ],
        ).v
        assert len(objs) == 2
        dist[i] = np.linalg.norm(pos[i, 0, :] - pos[i, 1, :])
    return x, dist

In [ ]:
def metrics(actual_collision, estimate_collision):
    # https://en.wikipedia.org/wiki/Confusion_matrix
    tp = np.count_nonzero(actual_collision & estimate_collision)
    fn = np.count_nonzero(actual_collision & ~estimate_collision)
    fp = np.count_nonzero(~actual_collision & estimate_collision)
    tn = np.count_nonzero(~actual_collision & ~estimate_collision)
    assert (tp + fn + fp + tn
            == len(actual_collision)
            == len(estimate_collision))

    print("          Actual    ")
    print("           P N      ")
    print("E    +------+------+")
    print("s  P | %4d | %4d |" % (tp, fp))
    print("t  N | %4d | %4d |" % (fn, tn))
    print(".    +------+------+")

    ppv = 100 * tp / (tp + fp)
    tpr = 100 * tp / (tp + fn)
    print("Precision : %1.d%%" % ppv)
    print("   Recall : %.1d%%" % tpr)
    print(" F1 score : %.1d%%" % (2 * ppv * tpr / (ppv + tpr)))

In [ ]:
train_x, train_dist = generate_collision_data(
    ssp_map.voc, names, -map_radius, map_radius, n_samples=10240)

In [ ]:
threshold = 2

In [ ]:
from sklearn import svm

clf = svm.SVC(C=5, kernel='rbf')
clf.fit(train_x, train_dist < threshold)

In [ ]:
metrics(actual_collision=(train_dist < threshold),
        estimate_collision=clf.predict(train_x))

In [ ]:
valid_x, valid_dist = generate_collision_data(
    ssp_map.voc, names, -map_radius, map_radius, n_samples=1024)

In [ ]:
metrics(actual_collision=(valid_dist < threshold),
        estimate_collision=clf.predict(valid_x))

In [ ]:
dt = 0.01
objp = [(-4, -3), (0, 5)]  # initial positions
objv = [(4, 3), (0, -5)]  # initial velocity

ground_ssp = []  # ground truth for each object
T = []  # translational shift for each object
for i in range(len(names)):
    ground_ssp.append(
        ssp_map.encode_point(objp[i][0], objp[i][1], name=names[i])
    )
    T.append(ssp_map.encode_point(dt*objv[i][0], dt*objv[i][1], name=None))

In [ ]:
sims = []

for step in range(100): 
    for i in range(len(ground_ssp)):
        ground_ssp[i] *= T[i]
        ground_ssp[i].name = ""  # https://github.com/nengo/nengo-spa/pull/246
    ssp = np.sum(ground_ssp)
    
    estimate_collision = clf.predict([ssp.v]).squeeze(axis=0)
    if estimate_collision:
        ssp = -ssp

    if step % 5 == 0:
        sim = np.sum(
            [ssp_map.compute_heatmap(ssp * ~ssp_map.voc[name])
             for name in names],
            axis=0)

        sims.append(sim)

ani = heatmap_animation([sims], figsize=(4, 4))
HTML('<img src="data:image/gif;base64,{0}" />'.format(create_gif(ani)))

In [ ]:
def lemniscate(t, a=4):
    # https://mathworld.wolfram.com/Lemniscate.html
    b = a * np.cos(t)
    return np.asarray([b, b * np.sin(t)]) / (1 + np.sin(t)**2)

# speech of each lemniscate
dt1 = 0.05
dt2 = 0.25
# radius of each lemniscate
a1 = 4
a2 = 3
# offset for each lemniscate
off1 = np.asarray([0, 0])
off2 = np.asarray([-1, -2])
# rotation matrix for each lemniscate
R1 = np.asarray([[0, 1], [1, 0]])
R2 = np.asarray([[1, 0], [0, 1]])

sims = []
for step in range(126):
    ssp = (
        ssp_map.encode_point(
            *R1.dot(lemniscate(t=dt1*step, a=a1) + off1), name=names[0])
        + ssp_map.encode_point(
            *R2.dot(lemniscate(t=dt2*step, a=a2) + off2), name=names[1])
    )

    estimate_collision = clf.predict([ssp.v]).squeeze(axis=0)
    if estimate_collision:
        ssp = -ssp

    if step % 1 == 0:
        sim = np.sum(
            [ssp_map.compute_heatmap(ssp * ~ssp_map.voc[name])
             for name in names],
            axis=0)

        sims.append(sim)

ani = heatmap_animation([sims], figsize=(4, 4))
HTML('<img src="data:image/gif;base64,{0}" />'.format(create_gif(ani)))